# Data Aggregation and Group Operations

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use ('ggplot')

## GroupBy mechanics

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})
df

,data1,data2,key1,key2
0,-1.216121,0.609964,a,one
1,1.364981,0.414077,a,two
2,2.120671,-0.431115,b,one
3,0.635128,0.302937,b,two
4,0.523093,1.208244,a,one


In [6]:
gb = df.groupby('key1').mean()
gb

,data1,data2
key1,,
a,0.223984,0.744095
b,1.377900,-0.064089


In [10]:
means = df.groupby('key1').mean()
means

,data1,data2
key1,,
a,0.223984,0.744095
b,1.377900,-0.064089


In [15]:
df.groupby(['key1','key2']).groups

{('a', 'one'): Int64Index([0, 4], dtype='int64'),
 ('a', 'two'): Int64Index([1], dtype='int64'),
 ('b', 'one'): Int64Index([2], dtype='int64'),
 ('b', 'two'): Int64Index([3], dtype='int64')}

In [12]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    1.364981
            2006    2.120671
Ohio        2005   -0.290496
            2006    0.523093
Name: data1, dtype: float64

### Iterating over groups

In [8]:
gb

NameError: name 'gb' is not defined

### Selecting a column or subset of columns

In [16]:
gb ['data1'].mean()

NameError: name 'gb' is not defined

## Data aggregation

In [17]:
df

,data1,data2,key1,key2
0,-0.454651,-0.430214,a,one
1,-0.783988,0.466550,a,two
2,0.960708,-0.317403,b,one
3,1.119551,1.017489,b,two
4,-2.459058,-1.886730,a,one


In [20]:
df.groupby('key1').mean()['data1']

key1
a   -1.232566
b    1.040130
Name: data1, dtype: float64

In [21]:
df.groupby('key1')['data1'].mean() # es diferente al anterior

key1
a   -1.232566
b    1.040130
Name: data1, dtype: float64

In [24]:
df.groupby('key1')['data1'].describe() # estadisticas

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,3.0,-1.232566,1.074862,-2.459058,-1.621523,-0.783988,-0.619320,-0.454651
b,2.0,1.040130,0.112319,0.960708,1.000419,1.040130,1.079841,1.119551


In [29]:
def max_to_min(series):
    return series.max()-series.min()

df.groupby('key2')['data1'].agg(max_to_min)


key2
one    3.419766
two    1.903540
Name: data1, dtype: float64

In [17]:
df

,data1,data2,key1,key2
0,-1.216121,0.609964,a,one
1,1.364981,0.414077,a,two
2,2.120671,-0.431115,b,one
3,0.635128,0.302937,b,two
4,0.523093,1.208244,a,one


In [19]:
df.groupby(['key1', 'key2'])[['data1']].mean()

data1
key1 key2          
a    one  -0.346514
     two   1.364981
b    one   2.120671
     two   0.635128

### Column-wise and multiple function application

In [16]:
for name1, group1 in df.groupby('key1'):
    print(name1)
    print(group1)
    

a
      data1     data2 key1 key2
0 -1.216121  0.609964    a  one
1  1.364981  0.414077    a  two
4  0.523093  1.208244    a  one
b
      data1     data2 key1 key2
2  2.120671 -0.431115    b  one
3  0.635128  0.302937    b  two


## Group-wise operations and transformations

### Apply: General split-apply-combine

#### Suppressing the group keys

### Quantile and bucket analysis

### Example: Filling missing values with group-specific values

In [55]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -0.607638
New York     -0.195258
Vermont            NaN
Florida       0.422648
Oregon        2.001614
Nevada             NaN
California   -1.104685
Idaho              NaN
dtype: float64

In [20]:
!wget https://github.com/wesm/pydata-book/raw/1st-edition/ch08/tips.csv

--2018-01-23 20:24:20--  https://github.com/wesm/pydata-book/raw/1st-edition/ch08/tips.csv
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wesm/pydata-book/1st-edition/ch08/tips.csv [following]
--2018-01-23 20:24:21--  https://raw.githubusercontent.com/wesm/pydata-book/1st-edition/ch08/tips.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.60.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.60.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1731 (1.7K) [text/plain]
Saving to: ‘tips.csv’

tips.csv            100%[===================>]   1.69K  --.-KB/s    in 0s      

2018-01-23 20:24:21 (12.0 MB/s) - ‘tips.csv’ saved [7943]



In [35]:
tips = pd.read_csv('tips.csv')
tips['Tip_porcen'] = tips['tip'] / tips['total_bill']
# Add tip percentage of total bill
tips[2:3]

,total_bill,tip,sex,smoker,day,time,size,Tip_porcen
2,21.01,3.5,Male,No,Sun,Dinner,3,0.166587


In [44]:
tips = pd.read_csv('tips.csv')
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


In [45]:
grouped = tips.groupby(['sex', 'smoker'])
grouped.mean()

total_bill       tip      size   tip_pct
sex    smoker                                          
Female No       18.105185  2.773519  2.592593  0.156921
       Yes      17.977879  2.931515  2.242424  0.182150
Male   No       19.791237  3.113402  2.711340  0.160669
       Yes      22.284500  3.051167  2.500000  0.152771

In [46]:
grouped_pct = grouped['tip_pct']
grouped_pct.mean()

sex     smoker
Female  No        0.156921
        Yes       0.182150
Male    No        0.160669
        Yes       0.152771
Name: tip_pct, dtype: float64

In [47]:

grouped['tip_pct', 'total_bill'].mean()

tip_pct  total_bill
sex    smoker                      
Female No      0.156921   18.105185
       Yes     0.182150   17.977879
Male   No      0.160669   19.791237
       Yes     0.152771   22.284500

In [49]:
np.var?

In [48]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill           
              Durchschnitt Abweichung Durchschnitt Abweichung
sex    smoker                                                
Female No         0.156921   0.001327    18.105185  53.092422
       Yes        0.182150   0.005126    17.977879  84.451517
Male   No         0.160669   0.001751    19.791237  76.152961
       Yes        0.152771   0.008206    22.284500  98.244673

In [51]:
grouped.agg({'tip_pct' : ['min', 'std'],
             'size' : 'sum'})

tip_pct           size
                    min       std  sum
sex    smoker                         
Female No      0.056797  0.036421  140
       Yes     0.056433  0.071595   74
Male   No      0.071804  0.041849  263
       Yes     0.035638  0.090588  150

## Pivot tables and Cross-tabulation